简单的 tokens_to_ids

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import re
import tensorflow as tf
import unicodedata
from collections import Counter
import random

In [2]:
tf.__version__

'2.2.0'

In [3]:
word_index = {}
special = []
with open('./vocab.txt') as fp:
    for i, line in enumerate(fp):
        line = line.strip() # .lower()
        word_index[line] = i
        if line.startswith('[') and line.endswith(']'):
            special.append(line)

print(len(special))
word_index[''] = word_index['[PAD]']
index_word = {v: k for k, v in word_index.items()}

104


In [4]:
special[:3]

['[PAD]', '[unused1]', '[unused2]']

In [5]:
special[-10:]

['[unused94]',
 '[unused95]',
 '[unused96]',
 '[unused97]',
 '[unused98]',
 '[unused99]',
 '[UNK]',
 '[CLS]',
 '[SEP]',
 '[MASK]']

In [6]:
'　' in word_index

False

In [8]:
'' in word_index

True

In [9]:
word_index[''] == word_index['[PAD]']

True

In [10]:
index_word[104]

'<S>'

In [11]:
index_word[101]

'[CLS]'

In [12]:
class BertToken2ids(tf.keras.models.Model):
    def __init__(self, word_index, **kwargs):
        super(BertToken2ids, self).__init__(**kwargs)
        self.construct(word_index)
    
    def construct(self, word_index):
        keys = tf.constant(list(word_index.keys()), dtype=tf.string)
        values = tf.constant(list(word_index.values()), dtype=tf.int32)
        self.table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(keys, values),
            tf.constant(word_index['[UNK]'])) # default value
    
    def call(self, inputs):
        x = inputs
        x = self.table.lookup(x)
        x = x[:, :512]
        return x

    def compute_output_shape(self, input_shape):
        return input_shape

In [13]:
t = BertToken2ids(word_index)

In [14]:
t(tf.ragged.constant(
    [['[unused1]', '[unused2]', '[unused3]'], ['[UNK]', '[CLS]', '[SEP]', '[MASK]']],
    dtype=tf.string
).to_tensor())

<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[  1,   2,   3,   0],
       [100, 101, 102, 103]], dtype=int32)>

In [15]:
t(tf.ragged.constant([
    ['a', 'a'],
    ['a'],
    ['[CLS]', '[SEP]'],
    ['[unused1]', '[unused2]', '[unused3]'],
    ['[UNK]', '[CLS]', '[SEP]', '[MASK]']
]).to_tensor())

<tf.Tensor: shape=(5, 4), dtype=int32, numpy=
array([[143, 143,   0,   0],
       [143,   0,   0,   0],
       [101, 102,   0,   0],
       [  1,   2,   3,   0],
       [100, 101, 102, 103]], dtype=int32)>

In [16]:
x = tf.ragged.constant([['a', 'a'], ['a']] * 250).to_tensor()

In [17]:
%%timeit
t(x)

214 µs ± 494 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [18]:
x.shape

TensorShape([500, 2])

In [20]:
save_path = 'bert_token2ids'

# model = tf.keras.Sequential([
#     BertTokenizer(word_index, name='bert_tokenizer'),
#     BertIds(word_index, name='bert_token_to_ids'),
# ])
# model.save(save_path, include_optimizer=False)
model = BertToken2ids(word_index)
# model._set_inputs(tf.keras.backend.placeholder((None, None), dtype=tf.int32))
model._set_inputs(
    tf.keras.backend.placeholder((None, None), dtype='string'))
model.save(save_path, include_optimizer=False)

INFO:tensorflow:Assets written to: bert_token2ids/assets


In [21]:
model.summary()

Model: "bert_token2ids_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [22]:
m2 = tf.keras.models.load_model(save_path)

In [23]:
text = [
    ['我', '爱', '你', '[MASK]', '[MASK]', '哦'],
    ['我', '[UNK]', '[MASK]'],
    ['important']
]
vec = tf.ragged.constant(text).to_tensor()

In [24]:
m2(vec)

<tf.Tensor: shape=(3, 6), dtype=int32, numpy=
array([[2769, 4263,  872,  103,  103, 1521],
       [2769,  100,  103,    0,    0,    0],
       [ 100,    0,    0,    0,    0,    0]], dtype=int32)>